# rUBotCoop Autonomous Navigation Laboratory Project

Autonomous navigation refers that the robot is able to move autonomously around the environment avoiding any obstacle.

In a hospital, a delivery robot carries samples or food from one room to another. 

The main objectives are:
- Create a real robot and its virtual model for simulation purposes
- locate the robot in our hospital environment
- perform autonomous navigation in the Hospital environment
- generate and store a map of the hospital 
- use SLAM (Simultaneous Localization and Mapping) techniques to find an optimal trajectory to reach a speciffic hospital target position

let's see how to fulfill these objectives

### ROS navigation packages
First, let's prepare your machine with the required ROS packages needed for the navigation stack:


In [ ]:
sudo apt install ros-melodic-navigation ros-melodic-amcl ros-melodic-map-server ros-melodic-move-base ros-melodic-urdf ros-melodic-xacro ros-melodic-compressed-image-transport ros-melodic-rqt-image-view

And finally the slam_gmapping package, that is already available in its binary version

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

Be sure about the ROS_HOSTNAME and ROS_MASTER_URI variables in the .bashrc file: (at home are)

export ROS_HOSTNAME=192.168.18.52
export ROS_MASTER_URI=http://192.168.18.52:11311

## rUBotCoop model generation

To perform autonomous Navigation, different sensors are needed and installed in the robot:
- a two-dimensional camera: correspondas to RBPi camera
- a Laser Distance Sensor (LDS): is the unidirectional distance sensor of the GoPiGo3 kit
- a 360º LIDAR sensor: corresponds to the EAI YDLIDAR X4 (https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html)

We have created a speciffic ROS Package to define a complete model of rUBotCoop robot including these sensors. This package is called "gopigo3_description" and the robot model is saved in URDF folder as "gopigo3.gazebo" file.

Launch the ROS visualization tool to check that the model is properly built. This tool is RViz and only represents the robot visual features. You have available all the options to check every aspect of the appearance of the model:

In [ ]:
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/LP01_full_model_rviz.png">

The code contains two new ROS packages as follows: 
- gopigo3_description, which contains the URDF model plus the SDF (Gazebo tags) for a complete, dynamic simulation. This package provides the gopigo3_rviz.launch launch file to interactively visualize the model in RViz.
- virtual_slam contains the virtual robot simulation itself, plus the launch files needed to run SLAM in Gazebo.

Then, rebuild the workspace so that it is known to your ROS installation:

## rUBotCoop in world environment

In robotics research, always before working with a real robot, we simulate the robot behaviour in a virtual environment close to the real one. The dynamic simulation of a robot, which, conceptually, is a better approach to examining the actual behavior of the robot rather than just using software. Rigid body mechanics, including mass and inertia, friction, damping, motor controllers, sensor detection properties, noise signals, and every aspect of the robot and the environment that can be retained in a model with reasonable accuracy is much less expensive when replicated in a simulator than if you tried to do this with physical hardware.

Gazebo is an open source 3D robotics simulator and includes an ODE physics engine and OpenGL rendering, and supports code integration for closed-loop control in robot drives. This is sensor simulation and actuator control.

A speciffic ROS Package called "gazebo_control" have been created for this purpose.

In [ ]:
roslaunch gazebo_control spawn.launch
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/LP02_rubotcoop_spawn.png">

<img src="./Images/LP02_rubotcoop_spawn_rviz.png">

In order to kill the previous Gazebo process, type:

killall gzserver && killall gzclient

### Design the Project world

Here we have first to design the project world, for exemple a hospital floor with different rooms where our rUBotCoop delivery robot has to navigate autonomously.

There is a very useful and simple tool to design a proper world: Building editor" in gazebo.

Open gazebo as superuser:

In [ ]:
sudo gazebo

You can build your world using "Building Editor" i Edit menu

<img src="./Images/LP03_BuildingWorld1_1.png">

You can save the generated world (world1.world) in the world folder.

Now, spawn the rUBotCoop robot in our generated world:

In [ ]:
roslaunch rubot_control rubot_navigation_world1.launch

<img src="./Images/LP04_BuildingWorld1_2.png">

Open RVIZ to see the sensors readings and add the laser_scan sensor:

In [ ]:
roslaunch rubot_control rubot_rviz.launch

Control the robot with the Keyboard

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

To test this sensor, it is better to use a Python script that makes the robot wander in the environment while avoiding the obstacles. To do this, we have implemented the following rules in our script: 
- If there is no obstacle, move forward at a reference speed of 0.8 m/s. 
- If the range provided by the distance sensor is lower than 2 meters, go back and rotate counter-clockwise until avoiding the obstacle. 
- Since the distance sensor throws unidirectional measurements, we should check the measurements from the LDS to find if there are obstacles to the sides, and the threshold should be lower than 1.6 meters. If obstacles are detected, go back and rotate counter-clockwise faster to avoid the obstacle and not get stuck on it.

This simple algorithm is implemented in the wanderAround.py script, and can be found under the ./virtual_slam/scripts/wanderAround.py folder.

First Kill the previous Gazebo process:
killall gzserver && killall gzclient

In [ ]:
rosrun rubot_control rubot_autonomous_navigation.py

<img src="./Images/01_Navigation_key.png">